# Practical 1 - Baysian Networks

### Monty Hall Problem

- Guests choice: $G = \{a, b, c\}$
- Prize door: $P = \{a, b, c\}$
- Monty's choice: $M = \{a, b, c\}$

In [1]:
import math
from pomegranate import *

# probability of guest choice
p_guest_choice = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})

# probability of prize door
p_prize_door = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})

#conditional probabilty of montys choice given guests choice and prize door
#[guest choice , prize door, monty's pick, probabilty]
cp_montys_choice = ConditionalProbabilityTable(
[
    [ 'A', 'A', 'A', 0.0 ],
    [ 'A', 'A', 'B', 0.5 ],
    [ 'A', 'A', 'C', 0.5 ],
    [ 'A', 'B', 'A', 0.0 ],
    [ 'A', 'B', 'B', 0.0 ],
    [ 'A', 'B', 'C', 1.0 ],
    [ 'A', 'C', 'A', 0.0 ],
    [ 'A', 'C', 'B', 1.0 ],
    [ 'A', 'C', 'C', 0.0 ],
    [ 'B', 'A', 'A', 0.0 ],
    [ 'B', 'A', 'B', 0.0 ],
    [ 'B', 'A', 'C', 1.0 ],
    [ 'B', 'B', 'A', 0.5 ],
    [ 'B', 'B', 'B', 0.0 ],
    [ 'B', 'B', 'C', 0.5 ],
    [ 'B', 'C', 'A', 1.0 ],
    [ 'B', 'C', 'B', 0.0 ],
    [ 'B', 'C', 'C', 0.0 ],
    [ 'C', 'A', 'A', 0.0 ],
    [ 'C', 'A', 'B', 1.0 ],
    [ 'C', 'A', 'C', 0.0 ],
    [ 'C', 'B', 'A', 1.0 ],
    [ 'C', 'B', 'B', 0.0 ],
    [ 'C', 'B', 'C', 0.0 ],
    [ 'C', 'C', 'A', 0.5 ],
    [ 'C', 'C', 'B', 0.5 ],
    [ 'C', 'C', 'C', 0.0 ],
], [p_guest_choice, p_prize_door])

#Definition of states
guest_choice_state = State(p_guest_choice, name="guest_choice_state")
prize_door_state = State(p_prize_door, name="prize_door_state")
montys_choice_state = State(cp_montys_choice, name="montys_choice_state")

#Building the Bayesian Network
network = BayesianNetwork("Solving the Monty Hall Problem with bayesian network")

network.add_states(guest_choice_state, prize_door_state, montys_choice_state)

network.add_edge(guest_choice_state, montys_choice_state)
network.add_edge(prize_door_state, montys_choice_state)

network.bake()

| $\mathrm{P}(P, M|G=a)$  | $a$ | $b$ | $c$ |
| ----------------------- | ----| --- | --- |
| $P$ | $\frac{1}{3}$ | $\frac{1}{3}$ | $\frac{1}{3}$ |
| $M$ | $0$ | $\frac{1}{2}$ | $\frac{1}{2}$ |

In [2]:
def print_proba(assignments, model):
    beliefs = model.predict_proba(assignments)
    beliefs = map(str, beliefs)
    print("\n".join("{}\t{}".format(state.name, belief) for state, belief in zip(model.states, beliefs)))

print_proba({'guest_choice_state' : 'A'}, network)

guest_choice_state	A
prize_door_state	{
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "A" : 0.3333333333333333,
            "B" : 0.3333333333333333,
            "C" : 0.3333333333333333
        }
    ],
    "frozen" : false
}
montys_choice_state	{
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "C" : 0.49999999999999983,
            "A" : 0.0,
            "B" : 0.49999999999999983
        }
    ],
    "frozen" : false
}


In [3]:
print_proba({'guest_choice_state' : 'A', 'montys_choice_state' : 'B'}, network)

guest_choice_state	A
prize_door_state	{
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "A" : 0.3333333333333334,
            "B" : 0.0,
            "C" : 0.6666666666666664
        }
    ],
    "frozen" : false
}
montys_choice_state	B


| $\mathrm{P}(P|G=a, M=b)$ | $a$ | $b$ | $c$ |
| ------------------------ | --- | --- | --- |
| $P$ | $\frac{1}{3}$ | $0$ | $\frac{2}{3}$ |

The initial guess by the guest has a $\frac{1}{3}$ chance of picking the price door. After Monty opens another door however the guest has additional information, since Monty choice is uniformaly random but conditioned on the actual pricedoor and the guests choice. This enables the guest to use this new information to make a better prediction. The CPT shows that after Monty reveals another Door, the Guest acts rationaly if they switch the remaining door.

### Burglary

The following are booleans
- Earthquake: $E$
- Burglary: $B$
- Alarm: $A$
- John calls: $J$
- Mary calls: $M$

In [4]:
B_prob = DiscreteDistribution({'True': 0.001, 'False': 0.999})
E_prob = DiscreteDistribution({'True': 0.002, 'False': 0.998})

# P(A|B, E)
A_prob = ConditionalProbabilityTable(
[
    ['True',  'True',  'True',  0.95],
    ['True',  'True',  'False', 0.05],
    ['True',  'False', 'True',  0.94],
    ['True',  'False', 'False', 0.06],
    ['False', 'True',  'True',  0.29],
    ['False', 'True',  'False', 0.61],
    ['False', 'False', 'True',  0.001],
    ['False', 'False', 'False', 0.999],
], [B_prob, E_prob])

# P(J|A)
J_prob = ConditionalProbabilityTable(
[
    ['True',  'True',  0.9],
    ['True',  'False', 0.1],
    ['False', 'True',  0.05],
    ['False', 'False', 0.95],
], [A_prob])

# P(M|A)
M_prob = ConditionalProbabilityTable(
[
    ['True',  'True',  0.7],
    ['True',  'False', 0.3],
    ['False', 'True',  0.01],
    ['False', 'False', 0.99],
], [A_prob])

# States 
B = State(B_prob, "Burglary")
E = State(E_prob, "Earthquake")
A = State(A_prob, "Alarm")
J = State(J_prob, "John calls")
M = State(M_prob, "Mary calls")

# Model
model = BayesianNetwork("Burglary Scenario")
model.add_states(B, E, A, J, M)
model.add_edge(B, A)
model.add_edge(E, A)
model.add_edge(A, J)
model.add_edge(A, M)
model.bake()

### Tasks:
#### 1. What is $\mathrm{P}(J \vee M | A)$?

Inclusion exclusion principle: 

$\because \mathrm{P}(J \vee M | A) = \mathrm{P}(J|A) + \mathrm{P}(M|A) - \mathrm{P}(J \wedge M|A)$

Hence $J$ and $M$ are conditionally independent given $A$:

$\because \mathrm{P}(J \wedge M|A) = \mathrm{P}(J|A)  \cdot \mathrm{P}(M|A)$

Therefore:

$\therefore \mathrm{P}(J \vee M | A) = \mathrm{P}(J|A) + \mathrm{P}(M|A) - \mathrm{P}(J|A)  \cdot \mathrm{P}(M|A) = 0.9 + 0.7 - 0.9 \cdot 0.7 = 0.97$

#### 2. What is $\mathrm{P}(J | \neg A)$?

CPT: $\mathrm{P}(J | \neg A) = 0.05$

#### 3. What is $\mathrm{P}(M, E)$?

$\mathrm{P}(M, E)= \text{A lot of math I am to lazy to write down} \approx 0.00042$

#### 4. What is $\mathrm{P}(A, \neg B, \neg E, \neg J, \neg M)$?

 $\mathrm{P}(A, \neg B, \neg E, \neg J, \neg M) = \mathrm{P}(A| \neg B, \neg E) \cdot \mathrm{P}(\neg J|A) \cdot \mathrm{P}(\neg M| A) \cdot \mathrm{P}(\neg B) \cdot \mathrm{P}(\neg E)$
 
 $0.001 \cdot 0.1 \cdot 0.3 \cdot 0.999 \cdot 0.998 \approx 0.0003$

In [6]:
def partial_probabilities(assignments, model):
    if all(isinstance(item, str) for item in assignments):
        return model.probability([assignments])
    else:
        for i, item in enumerate(assignments):
            if isinstance(item, list):
                return sum(partial_probabilities(assignments[:i] + [var] + assignments[i+1:], model) for var in item)



task_1 = partial_probabilities([
    ['True', 'False'], # Burglary
    ['True', 'False'], # Earthquake
    'True',            # Alarm
    'True',            # John calls
    ['True', 'False'], # Mary calls
], model) / partial_probabilities([
    ['True', 'False'], # Burglary
    ['True', 'False'], # Earthquake
    'True',            # Alarm
    ['True', 'False'], # John calls
    ['True', 'False'], # Mary calls
], model) + partial_probabilities([
    ['True', 'False'], # Burglary
    ['True', 'False'], # Earthquake
    'True',            # Alarm
    ['True', 'False'], # John calls
    'True',            # Mary calls
], model) / partial_probabilities([
    ['True', 'False'], # Burglary
    ['True', 'False'], # Earthquake
    'True',            # Alarm
    ['True', 'False'], # John calls
    ['True', 'False'], # Mary calls
], model) - partial_probabilities([
    ['True', 'False'], # Burglary
    ['True', 'False'], # Earthquake
    'True',            # Alarm
    'True',            # John calls
    ['True', 'False'], # Mary calls
], model) / partial_probabilities([
    ['True', 'False'], # Burglary
    ['True', 'False'], # Earthquake
    'True',            # Alarm
    ['True', 'False'], # John calls
    ['True', 'False'], # Mary calls
], model) * partial_probabilities([
    ['True', 'False'], # Burglary
    ['True', 'False'], # Earthquake
    'True',            # Alarm
    ['True', 'False'], # John calls
    'True',            # Mary calls
], model) / partial_probabilities([
    ['True', 'False'], # Burglary
    ['True', 'False'], # Earthquake
    'True',            # Alarm
    ['True', 'False'], # John calls
    ['True', 'False'], # Mary calls
], model)

task_2 = partial_probabilities([
    'False',           # Burglary
    ['True', 'False'], # Earthquake
    ['True', 'False'], # Alarm
    'True',            # John calls
    ['True', 'False'], # Mary calls
], model) / partial_probabilities([
    'False',           # Burglary
    ['True', 'False'], # Earthquake
    ['True', 'False'], # Alarm
    ['True', 'False'], # John calls
    ['True', 'False'], # Mary calls
], model)

task_3 = partial_probabilities([
    ['True', 'False'], # Burglary
    'True',            # Earthquake
    ['True', 'False'], # Alarm
    ['True', 'False'], # John calls
    'True',            # Mary calls
], model)

task_4 = partial_probabilities([
    'False', # Burglary
    'False', # Earthquake
    'True',  # Alarm
    'False', # John calls
    'False', # Mary calls
], model)

print(f"""Burglary Scenario:
__________________

What is the propability of John or Mary calling given alarm?
{round(task_1, 2)}

What is the propability of John calling, but there was no alarm?
{round(task_2, 2)}

What is the propability of Mary calling and there was an earthquake?
{round(task_3, 5)}

What is the propability of the alarm going off, but there was no burglary, no earthquake, and neither John nor Mary called?
{round(task_4, 5)}""")

Burglary Scenario:
__________________

What is the propability of John or Mary calling given alarm?
0.97

What is the propability of John calling, but there was no alarm?
0.05

What is the propability of Mary calling and there was an earthquake?
0.00042

What is the propability of the alarm going off, but there was no burglary, no earthquake, and neither John nor Mary called?
3e-05
